<img src="bank_marketing.webp" style="width: calc(100% - 10px); height: 400px; margin: 5px;">

## Personal Loan Campaign Data Cleaning

### Project Overview

Personal loans are a key revenue source for banks. In the UK, the average interest rate on a two-year personal loan is around 10%. In September 2022 alone, UK consumers borrowed **£1.5 billion**, generating about **£300 million in interest** over two years.

In this project, you’ll clean and prepare data from a marketing campaign aimed at increasing personal loan sign-ups. The bank intends to use this data for future campaigns, so the cleaned data must adhere to a specific structure and set of data types.

You are provided with the CSV file `"bank_marketing.csv"`, which you’ll clean, reformat, and split into three separate CSV files for further use. 


In [14]:
import pandas as pd
import numpy as np


# Step 1: Load the CSV File
data = pd.read_csv("Input_Data/bank_marketing.csv")

# Step 2: Understand the Data
print(data)

       client_id  age          job  marital            education  \
0              0   56    housemaid  married             basic.4y   
1              1   57     services  married          high.school   
2              2   37     services  married          high.school   
3              3   40       admin.  married             basic.6y   
4              4   56     services  married          high.school   
...          ...  ...          ...      ...                  ...   
41183      41183   73      retired  married  professional.course   
41184      41184   46  blue-collar  married  professional.course   
41185      41185   56      retired  married    university.degree   
41186      41186   44   technician  married  professional.course   
41187      41187   74      retired  married  professional.course   

      credit_default mortgage month  day  contact_duration  number_contacts  \
0                 no       no   may   13               261                1   
1            unknown     

In [15]:
print(data.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')


In [16]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [17]:
# Step 3: Split the Data
# Define the columns for each DataFrame based on requirements
client_columns = ["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]
campaign_columns = ["client_id", "number_contacts", "month", "day","contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]
economics_columns = ["client_id", "cons_price_idx", "euribor_three_months"]

# Create separate DataFrames
client = data[client_columns].copy()
campaign = data[campaign_columns].copy()
economics = data[economics_columns].copy()

In [18]:
# step 4 : Clean and Format

# Editing the client dataset

# Clean job column
client["job"] = client["job"].str.replace(".", "_")
# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Clean and convert client columns to bool data type
for col in ["credit_default", "mortgage"]:
    client[col] = client[col].apply(lambda x: 1 if x == "yes" else 0)
    client[col] = client[col].astype(bool)


In [19]:
print(client)

       client_id  age          job  marital            education  \
0              0   56    housemaid  married             basic_4y   
1              1   57     services  married          high_school   
2              2   37     services  married          high_school   
3              3   40       admin_  married             basic_6y   
4              4   56     services  married          high_school   
...          ...  ...          ...      ...                  ...   
41183      41183   73      retired  married  professional_course   
41184      41184   46  blue-collar  married  professional_course   
41185      41185   56      retired  married    university_degree   
41186      41186   44   technician  married  professional_course   
41187      41187   74      retired  married  professional_course   

       credit_default  mortgage  
0               False     False  
1               False     False  
2               False      True  
3               False     False  
4            

In [20]:
# Print column names and data types to understand structure
print(client.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None


In [21]:
# Editing the campaign dataset
# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Convert previous_outcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0,
                                                                 "nonexistent": 0})

# Add year column
campaign["year"] = "2022"

# Convert day to string
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

# Clean and convert outcome columns to bool
for col in ["campaign_outcome", "previous_outcome"]:
  campaign[col] = campaign[col].astype(bool)

# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)


In [22]:
print(campaign)

       client_id  number_contacts  contact_duration  \
0              0                1               261   
1              1                1               149   
2              2                1               226   
3              3                1               151   
4              4                1               307   
...          ...              ...               ...   
41183      41183                1               334   
41184      41184                1               383   
41185      41185                2               189   
41186      41186                1               442   
41187      41187                3               239   

       previous_campaign_contacts  previous_outcome  campaign_outcome  \
0                               0             False             False   
1                               0             False             False   
2                               0             False             False   
3                               0             F

In [23]:
# Print column names and data types to understand structure
print(campaign.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB
None


In [24]:
print(economics)

       client_id  cons_price_idx  euribor_three_months
0              0          93.994                 4.857
1              1          93.994                 4.857
2              2          93.994                 4.857
3              3          93.994                 4.857
4              4          93.994                 4.857
...          ...             ...                   ...
41183      41183          94.767                 1.028
41184      41184          94.767                 1.028
41185      41185          94.767                 1.028
41186      41186          94.767                 1.028
41187      41187          94.767                 1.028

[41188 rows x 3 columns]


In [25]:
# Print column names and data types to understand structure
print(economics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None


In [26]:
# Step 5: Save the DataFrames
client.to_csv("Output_Data/client.csv", index=False)
campaign.to_csv("Output_Data/campaign.csv", index=False)
economics.to_csv("Output_Data/economics.csv", index=False)

print("Files saved successfully!")

Files saved successfully!
